In [1]:
import numpy as np


def generate_time_series(batch_size, n_steps):
    """
    随机生成固定长度的序列样本。
    序列由固定振幅，相位和频率随机的两个正弦波和随机噪音加和而成。
    序列样本一般表示为 3D 数组 [batch size, time steps, dimensionality], 对单变量 dimensionality 为 1
    Parameters
    ----------
    batch_size 生成时间序列个数（样本数）
    n_steps 序列长度
    每个时间步一个值，即单变量序列
    Returns
    -------

    """
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  # wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20))  # wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)  # noise
    return series[..., np.newaxis].astype(np.float32)

创建训练集、验证集和测试集

In [2]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

X_train 包含 7,000 条序列数据，其 shape 为 [7000, 500, 1]，
X_valid 包含 2,000 条序列数据，从 7,000th 到 8,999th
X_test 包含 1,000 条序列数据，从 9,000th 到 9,999th

由于对每个序列只预测一个值，所以目标值为列向量，例如 y_train 的 shape 为 [7000, 1]

## 基线指标

在开始使用 RNN 之前，最好先定义一些基线指标，否则我们认为我们的模型很好，但实际上它比基本模型都差。例如，最简单的方法是预测每个序列的最后一个值。这就是所谓的朴素预测（naive forecasting），有时候要超越它十分困难。


In [8]:
y_pred = X_valid[:, -1]  # 取 X 的最后一个值，作为预测的 y 值

In [9]:
import tensorflow.keras as keras

In [10]:
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))

0.020602005

平均平方误差为 0.020。

另一种方法是使用全连接网络。因为全连接需要一维数据，所以添加一个 `Flatten` 层。下面使用一个简单的线性回归模型，这样每个预测都将是时间序列中值的线性组合：

In [11]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50, 1]),
    keras.layers.Dense(1)
])

下面我们以 MSE loss 和 Adam 优化器编译模型，训练 20 个 epochs，并在验证集上评估

In [12]:
model.compile(optimizer='adam',
              loss='mse')

In [13]:
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
219/219 [==============================] - 2s 2ms/step - loss: 0.1685
Epoch 2/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0296
Epoch 3/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0151
Epoch 4/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0105
Epoch 5/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0088
Epoch 6/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0079
Epoch 7/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0072
Epoch 8/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0066
Epoch 9/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0061
Epoch 10/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0056
Epoch 11/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0052
Epoch 12/20
219/219 [==============================] - 0s 2ms/step - loss: 0.0049
Epoch 13/20
219/219 [====

In [14]:
model.evaluate(X_valid, y_valid)

63/63 [==============================] - 0s 1ms/step - loss: 0.0035


0.003472333773970604

损失值为 0.003，比 naive 方法好。

In [15]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

上面是最简单的 RNN，只包含一层，该层只有一个 neuron。

并且不需要指定输入序列长度，因为 RNN 可以处理任意长度序列。

其初始状态 $h_{(init)}$ 设置为 0，和初始值 $x_{(0)}$ 传递给唯一的 neuron，该 neuron 计算加权和，应用 tanh 激活函数，获得输出 $y_0$。在 simple RNN，该输出还作为新的状态 $h_0$ 使用。该状态和 $x_{(1)}$ 再次传递给 neuron，反复执行，直到最后一步。最后一层输出 $y_{49}$ 。

In [16]:
model.compile(optimizer='adam',
              loss="mse")

In [17]:
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
219/219 [==============================] - 13s 59ms/step - loss: 0.2717
Epoch 2/20
219/219 [==============================] - 13s 60ms/step - loss: 0.1429
Epoch 3/20
219/219 [==============================] - 13s 60ms/step - loss: 0.0684
Epoch 4/20
219/219 [==============================] - 12s 55ms/step - loss: 0.0470
Epoch 5/20
219/219 [==============================] - 11s 52ms/step - loss: 0.0408
Epoch 6/20
219/219 [==============================] - 11s 51ms/step - loss: 0.0367
Epoch 7/20
219/219 [==============================] - 12s 57ms/step - loss: 0.0333
Epoch 8/20
219/219 [==============================] - 13s 60ms/step - loss: 0.0304
Epoch 9/20
219/219 [==============================] - 12s 53ms/step - loss: 0.0278
Epoch 10/20
219/219 [==============================] - 13s 58ms/step - loss: 0.0256
Epoch 11/20
219/219 [==============================] - 13s 60ms/step - loss: 0.0237
Epoch 12/20
219/219 [==============================] - 13s 60ms/step - loss: 0.0220
E

MSE 只到 0.014，比 naive 方法好点，但是没有简单的线性模型好。

而且训练时间明显更长。

不过应该要明白，线性模型对每个输入都有一个参数，加上 bias 项，共 51 个参数。

而上面的 RNN 只有一个神经元，一个输入参数，一个 hidden 参数，一个 bias 想，只有 3 个参数。

## Deep RNNs

下面使用深度循环神经网络，使用 `tf.keras` 实现 deep RNN 十分简单，叠加 RNN 即可。在这里我们使用 3 个 `SimpleRNN`，也可以使用其它 RNN 层，如 `LSTM` 或 `GRU`。

In [18]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

除了最后一层，中间的循环层都要设置 `return_sequences=True`，否则循环层只输出 2D 数组，而不是 RNN 层所需的 3D 数组。

In [19]:
model.compile(optimizer='adam',
              loss='mse')

In [20]:
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
219/219 [==============================] - 46s 204ms/step - loss: 0.0385
Epoch 2/20
219/219 [==============================] - 45s 205ms/step - loss: 0.0063
Epoch 3/20
219/219 [==============================] - 46s 209ms/step - loss: 0.0051
Epoch 4/20
219/219 [==============================] - 45s 208ms/step - loss: 0.0046
Epoch 5/20
219/219 [==============================] - 45s 207ms/step - loss: 0.0040
Epoch 6/20
219/219 [==============================] - 45s 206ms/step - loss: 0.0038
Epoch 7/20
219/219 [==============================] - 44s 203ms/step - loss: 0.0036
Epoch 8/20
219/219 [==============================] - 45s 204ms/step - loss: 0.0036
Epoch 9/20
219/219 [==============================] - 45s 203ms/step - loss: 0.0034
Epoch 10/20
219/219 [==============================] - 41s 188ms/step - loss: 0.0033
Epoch 11/20
219/219 [==============================] - 41s 187ms/step - loss: 0.0034
Epoch 12/20
219/219 [==============================] - 42s 192ms/step - lo

上面的深度 RNN，最后 MSE 达到 0.003，比线性模型好了。

最后一层使用单变量的RNN并不合适。将其替换为 `Dense` 更好，速度会稍微快点，精确差不多，并且可以选择自己需要的激活函数。

如果最后一个层使用 `Dense`，需要移除上一层的 `return_sequences=True` 设置。

In [ ]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
])